# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load smsemoa results for all topos

In [2]:
with open('res_bc_Cogent_smsemoa.pkl','rb') as f_Cogent:
    res_8_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_smsemoa.pkl','rb') as f_UsCarrier:
    res_8_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_smsemoa.pkl','rb') as f_HiberniaGlobal:
    res_8_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_smsemoa.pkl','rb') as f_Colt:
    res_8_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_smsemoa.pkl','rb') as f_Funet:
    res_8_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_smsemoa.pkl','rb') as f_Abvt:
    res_8_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_smsemoa.pkl','rb') as f_Intellifiber:
    res_8_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_smsemoa.pkl','rb') as f_TataNld:
    res_8_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_smsemoa.pkl','rb') as f_Internode:
    res_8_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_smsemoa.pkl','rb') as f_Missouri:
    res_8_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_smsemoa.pkl','rb') as f_Ion:
    res_8_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_smsemoa.pkl','rb') as f_Palmetto:
    res_8_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# smsemoa
F8_Cogent=res_8_Cogent.F
F8_UsCarrier=res_8_UsCarrier.F
F8_HiberniaGlobal=res_8_HiberniaGlobal.F
F8_Colt=res_8_Colt.F
F8_Funet=res_8_Funet.F
F8_Abvt=res_8_Abvt.F
F8_Intellifiber=res_8_Intellifiber.F
F8_TataNld=res_8_TataNld.F
F8_Internode=res_8_Internode.F
F8_Missouri=res_8_Missouri.F
F8_Ion=res_8_Ion.F
F8_Palmetto=res_8_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F8_Cogent = []
hist_F8_UsCarrier = []
hist_F8_HiberniaGlobal = []
hist_F8_Colt = []
hist_F8_Funet = []
hist_F8_Abvt = []
hist_F8_Intellifiber = []
hist_F8_TataNld = []
hist_F8_Internode = []
hist_F8_Missouri = []
hist_F8_Ion = []
hist_F8_Palmetto = []


for algo8_Cogent in res_8_Cogent.history:
    opt8_Cogent = algo8_Cogent.opt
    feas8_Cogent = np.where(opt8_Cogent.get("feasible"))[0]
    hist_F8_Cogent.append(opt8_Cogent.get("F")[feas8_Cogent])
for algo8_UsCarrier in res_8_UsCarrier.history:
    opt8_UsCarrier = algo8_UsCarrier.opt
    feas8_UsCarrier = np.where(opt8_UsCarrier.get("feasible"))[0]
    hist_F8_UsCarrier.append(opt8_UsCarrier.get("F")[feas8_UsCarrier])
for algo8_HiberniaGlobal in res_8_HiberniaGlobal.history:
    opt8_HiberniaGlobal = algo8_HiberniaGlobal.opt
    feas8_HiberniaGlobal = np.where(opt8_HiberniaGlobal.get("feasible"))[0]
    hist_F8_HiberniaGlobal.append(opt8_HiberniaGlobal.get("F")[feas8_HiberniaGlobal])
for algo8_Colt in res_8_Colt.history:
    opt8_Colt = algo8_Colt.opt
    feas8_Colt = np.where(opt8_Colt.get("feasible"))[0]
    hist_F8_Colt.append(opt8_Colt.get("F")[feas8_Colt])
for algo8_Funet in res_8_Funet.history:
    opt8_Funet = algo8_Funet.opt
    feas8_Funet = np.where(opt8_Funet.get("feasible"))[0]
    hist_F8_Funet.append(opt8_Funet.get("F")[feas8_Funet])
for algo8_Abvt in res_8_Abvt.history:
    opt8_Abvt = algo8_Abvt.opt
    feas8_Abvt = np.where(opt8_Abvt.get("feasible"))[0]
    hist_F8_Abvt.append(opt8_Abvt.get("F")[feas8_Abvt])
for algo8_Intellifiber in res_8_Intellifiber.history:
    opt8_Intellifiber = algo8_Intellifiber.opt
    feas8_Intellifiber = np.where(opt8_Intellifiber.get("feasible"))[0]
    hist_F8_Intellifiber.append(opt8_Intellifiber.get("F")[feas8_Intellifiber])
for algo8_TataNld in res_8_TataNld.history:
    opt8_TataNld = algo8_TataNld.opt
    feas8_TataNld = np.where(opt8_TataNld.get("feasible"))[0]
    hist_F8_TataNld.append(opt8_TataNld.get("F")[feas8_TataNld])
for algo8_Internode in res_8_Internode.history:
    opt8_Internode = algo8_Internode.opt
    feas8_Internode= np.where(opt8_Internode.get("feasible"))[0]
    hist_F8_Internode.append(opt8_Internode.get("F")[feas8_Internode])
for algo8_Missouri in res_8_Missouri.history:
    opt8_Missouri = algo8_Missouri.opt
    feas8_Missouri = np.where(opt8_Missouri.get("feasible"))[0]
    hist_F8_Missouri.append(opt8_Missouri.get("F")[feas8_Missouri])
for algo8_Ion in res_8_Ion.history:
    opt8_Ion = algo8_Ion.opt
    feas8_Ion = np.where(opt8_Ion.get("feasible"))[0]
    hist_F8_Ion.append(opt8_Ion.get("F")[feas8_Ion])
for algo8_Palmetto in res_8_Palmetto.history:
    opt8_Palmetto = algo8_Palmetto.opt
    feas8_Palmetto = np.where(opt8_Palmetto.get("feasible"))[0]
    hist_F8_Palmetto.append(opt8_Palmetto.get("F")[feas8_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for smsemoa algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_8_Cogent = [metric.do(_F8_Cogent) for _F8_Cogent in hist_F8_Cogent]
hv_8_UsCarrier = [metric.do(_F8_UsCarrier) for _F8_UsCarrier in hist_F8_UsCarrier]
hv_8_HiberniaGlobal = [metric.do(_F8_HiberniaGlobal) for _F8_HiberniaGlobal in hist_F8_HiberniaGlobal]
hv_8_Colt = [metric.do(_F8_Colt) for _F8_Colt in hist_F8_Colt]
hv_8_Funet = [metric.do(_F8_Funet) for _F8_Funet in hist_F8_Funet]
hv_8_Abvt = [metric.do(_F8_Abvt) for _F8_Abvt in hist_F8_Abvt]
hv_8_Intellifiber = [metric.do(_F8_Intellifiber) for _F8_Intellifiber in hist_F8_Intellifiber]
hv_8_TataNld = [metric.do(_F8_TataNld) for _F8_TataNld in hist_F8_TataNld]
hv_8_Internode = [metric.do(_F8_Internode) for _F8_Internode in hist_F8_Internode]
hv_8_Missouri = [metric.do(_F8_Missouri) for _F8_Missouri in hist_F8_Missouri]
hv_8_Ion = [metric.do(_F8_Ion) for _F8_Ion in hist_F8_Ion]
hv_8_Palmetto = [metric.do(_F8_Palmetto) for _F8_Palmetto in hist_F8_Palmetto]

# smsemoa
hv_8 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_8_Cogent, hv_8_UsCarrier, hv_8_HiberniaGlobal, hv_8_Colt, hv_8_Funet, hv_8_Abvt, hv_8_Intellifiber, hv_8_TataNld, hv_8_Internode, hv_8_Missouri, hv_8_Ion, hv_8_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_8_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_8_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_8_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_8_Colt)
print('The hypervolumn for Funet:')
print(hv_8_Funet)
print('The hypervolumn for Abvt:')
print(hv_8_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_8_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_8_TataNld)
print('The hypervolumn for Internode:')
print(hv_8_Internode)
print('The hypervolumn for Missouri:')
print(hv_8_Missouri)
print('The hypervolumn for Ion:')
print(hv_8_Ion)
print('The hypervolumn for Palmetto:')
print(hv_8_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_8_Cogent)+1)), hv_8_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_8_UsCarrier)+1)), hv_8_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_8_HiberniaGlobal)+1)), hv_8_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_8_Colt)+1)), hv_8_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_8_Funet)+1)), hv_8_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_8_Abvt)+1)), hv_8_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_8_Intellifiber)+1)), hv_8_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_8_TataNld)+1)), hv_8_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_8_Internode)+1)), hv_8_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_8_Missouri)+1)), hv_8_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_8_Ion)+1)), hv_8_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_8_Palmetto)+1)), hv_8_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for smsemoa:')
print(hv_8)

3. Draw mean hypervolume for smsemoa

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()